In [1]:
import numpy as np

from transforms84.distances import Haversine
from transforms84.helpers import (
    DDM2RRM,
    RRM2DDM,
    deg_angular_difference,
)
from transforms84.systems import WGS84
from transforms84.transforms import ECEF2ENU, ENU2AER, geodetic2ECEF

## Coordinate Transformations

In [2]:
rrm_local = DDM2RRM(np.array([[30], [31], [0]], dtype=np.float64))
rrm_target = DDM2RRM(np.array([[31], [32], [0]], dtype=np.float64))

In [3]:
RRM2DDM(rrm_target)

array([[31.],
       [32.],
       [ 0.]])

In [4]:
rrm_local2target = ENU2AER(
    ECEF2ENU(rrm_local, geodetic2ECEF(rrm_target, WGS84.a, WGS84.b), WGS84.a, WGS84.b)
)
ddm_local2target = RRM2DDM(rrm_local2target)
ddm_local2target

array([[ 4.06379074e+01],
       [-6.60007585e-01],
       [ 1.46643956e+05]])

If we duplicate the local and target points three times each we will get an output of three local azimuth-elevation-range coordinates.

In [5]:
num_repeats = 3
rrm_targets = np.ascontiguousarray(
    np.tile(rrm_target, num_repeats).T.reshape((-1, 3, 1))
)
rrm_locals = np.ascontiguousarray(
    np.tile(rrm_local, rrm_targets.shape[0]).T.reshape((-1, 3, 1))
)
ddm_local2targets = RRM2DDM(
    ENU2AER(
        ECEF2ENU(
            rrm_locals, geodetic2ECEF(rrm_targets, WGS84.a, WGS84.b), WGS84.a, WGS84.b
        )
    )
)
assert np.all(np.unique(ddm_local2targets, axis=0)[0] == ddm_local2target)
ddm_local2targets

array([[[ 4.06379074e+01],
        [-6.60007585e-01],
        [ 1.46643956e+05]],

       [[ 4.06379074e+01],
        [-6.60007585e-01],
        [ 1.46643956e+05]],

       [[ 4.06379074e+01],
        [-6.60007585e-01],
        [ 1.46643956e+05]]])

Alternatively, we can keep the origin point fixed to one point and just duplicate the target points three times.

In [6]:
ddm_local2targets1 = RRM2DDM(
    ENU2AER(
        ECEF2ENU(
            rrm_local, geodetic2ECEF(rrm_targets, WGS84.a, WGS84.b), WGS84.a, WGS84.b
        )
    )
)
assert np.all(np.unique(ddm_local2targets1, axis=0)[0] == ddm_local2target)
ddm_local2targets1

array([[[ 4.06379074e+01],
        [-6.60007585e-01],
        [ 1.46643956e+05]],

       [[ 4.06379074e+01],
        [-6.60007585e-01],
        [ 1.46643956e+05]],

       [[ 4.06379074e+01],
        [-6.60007585e-01],
        [ 1.46643956e+05]]])

We can also use split up the input along the axes of the coordinate system to achieve the same result as above.

In [7]:
rad_lat_origin = np.ascontiguousarray(rrm_locals[:, 0, 0])
rad_lon_origin = np.ascontiguousarray(rrm_locals[:, 1, 0])
m_alt_origin = np.ascontiguousarray(rrm_locals[:, 2, 0])
rad_lat_targets = np.ascontiguousarray(rrm_targets[:, 0, 0])
rad_lon_targets = np.ascontiguousarray(rrm_targets[:, 1, 0])
m_alt_targets = np.ascontiguousarray(rrm_targets[:, 2, 0])
rad_az, rad_el, m_range = ENU2AER(
    *ECEF2ENU(
        rad_lat_origin,
        rad_lon_origin,
        m_alt_origin,
        *geodetic2ECEF(
            rad_lat_targets, rad_lon_targets, m_alt_targets, WGS84.a, WGS84.b
        ),
        WGS84.a,
        WGS84.b,
    )
)
np.rad2deg(rad_az), np.rad2deg(rad_el), m_range

(array([40.63790736, 40.63790736, 40.63790736]),
 array([-0.66000759, -0.66000759, -0.66000759]),
 array([146643.95639635, 146643.95639635, 146643.95639635]))

## Distance Calculation
Compare the above range to the Haversine method:

In [8]:
m_distance = Haversine(rrm_local, rrm_target, WGS84.mean_radius)
m_distance, m_distance - ddm_local2target[2, 0]

(146775.88330369865, np.float64(131.92690734518692))

In [9]:
Haversine(
    rrm_locals.astype(np.float32), rrm_targets.astype(np.float32), WGS84.mean_radius
)

array([146768.66, 146768.66, 146768.66], dtype=float32)

In [10]:
m_distances1 = Haversine(rrm_locals, rrm_targets, WGS84.mean_radius)
m_distances1

array([146775.8833037, 146775.8833037, 146775.8833037])

In [11]:
m_distances2 = Haversine(rrm_local, rrm_targets, WGS84.mean_radius)
m_distances2

array([146775.8833037, 146775.8833037, 146775.8833037])

In [12]:
m_distances2 = Haversine(
    np.ascontiguousarray(np.tile(rrm_local, 10).T.reshape((-1, 3, 1))),
    np.ascontiguousarray(np.tile(rrm_target, 10).T.reshape((-1, 3, 1))),
    (2 * 6378137.0 + 6356752.314245) / 3,
)
m_distances2

array([146775.8833037, 146775.8833037, 146775.8833037, 146775.8833037,
       146775.8833037, 146775.8833037, 146775.8833037, 146775.8833037,
       146775.8833037, 146775.8833037])

In [13]:
m_distance == np.unique(m_distances1)[0] == np.unique(m_distances2)[0]

np.True_

## Misc. Functions

In [14]:
(
    deg_angular_difference(50, 70, True),
    deg_angular_difference(70, 50, True),
    deg_angular_difference(70, 50, False),
)

(20.0, 20.0, 340.0)

In [15]:
(
    deg_angular_difference(
        np.array([50, 50], dtype=np.float32), np.array([70, 70], dtype=np.float32), True
    ),
    deg_angular_difference(
        np.array([70, 70], dtype=np.float32), np.array([50, 50], dtype=np.float32), True
    ),
    deg_angular_difference(
        np.array([70, 70], dtype=np.float32),
        np.array([50, 50], dtype=np.float32),
        False,
    ),
)

(array([20., 20.], dtype=float32),
 array([20., 20.], dtype=float32),
 array([340., 340.], dtype=float32))